In [1]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from pickle import load
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_yaml
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D
from numpy import array
from numpy import hstack
from math import floor
import toml


from ipynb.fs.full.V07_Utils  import *
from ipynb.fs.full.V07_Prepare_dataset import *

In [2]:
######################################################################
# Load Configuration
######################################################################

config_path = 'C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Code/V07_config.toml'
cfg = toml.load(config_path)


######################################################################
# Load Parameters 
######################################################################

# General
n_steps = cfg['n_steps']
LAT_in_rows = cfg['LAT_in_rows']

# Quats
n_in_seq_quat = cfg['n_in_seq_quat']
n_out_seq_quat = cfg['n_out_seq_quat']
n_features_quat = cfg['n_features_quat']

# Position
n_in_seq_pos = cfg['n_in_seq_pos']
n_out_seq_pos = cfg['n_out_seq_pos']
n_features_pos = cfg['n_features_pos']


######################################################################
# Load Directories 
######################################################################

# Dataset path 
Path_dataset01_test = cfg['Path_dataset01']

# GRU euler
Path_results_dataset01_GRU_euler = cfg['Path_results_dataset01'] + "GRU_euler/"

# GRU quat
Path_results_dataset01_GRU_quat = cfg['Path_results_dataset01'] + "GRU_quat/"

# GRU pos
Path_results_dataset01_GRU_pos = cfg['Path_results_dataset01'] + "GRU_pos/"

# Baseline quat
Path_results_dataset01_Baseline_quat = cfg['Path_results_dataset01'] + "Baseline_quat/"

# Baseline pos
Path_results_dataset01_Baseline_pos = cfg['Path_results_dataset01'] + "Baseline_pos/"

# Tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  0


# Multivariate Data Preparation

In [3]:
########################## 
# Baseline Position 
##########################

def baseline_pos(path_test, result_path, n_features, n_steps, LAT_in_rows):

    for csv in get_csv_files(path_test, "full_cont"):
            print(csv)
            basename = os.path.splitext(os.path.basename(csv))[0]
            
            preds_pos = []
            true_pos = []
            
            X_test, y_test = prepare_testing_data_pos_special(csv)     
            for i in range(len(X_test)):
                x_input = X_test[i]
                
                # Make single prediction and load single groundtruth 
                phat = x_input[-1]
                ptrue = y_test[i]

                # Store single prediction and corresponding groundtruth in list
                preds_pos.append(rhat)
                true_pos.append(rtrue)

            # Create an array 
            preds_pos = array(preds_pos)
            true_pos = array(true_pos)
            true_pos = true_pos.reshape((true_pos.shape[0],1, true_pos.shape[1]))

            # Create dataframe for prediction & gt per timestamp and concate results 
            df_preds_pos = pd.DataFrame(np.row_stack(preds_pos), columns= cfg['pos_coords_pred'])
            df_true_pos = pd.DataFrame(np.row_stack(true_pos), columns= cfg['pos_coords_true'])
            df_pos_result = pd.concat([df_preds_pos, df_true_pos], axis=1)

            # Save result to csv
            df_rot_result.to_csv(os.path.join(result_path, basename + '_result_pos.csv'), index=False)
            print("Single Predictions are saved")

In [4]:
########################## 
# Baseline Position Delta
##########################

def baseline_delta_pos(path_test, result_path, n_features, n_steps, LAT_in_rows):
    
    
     for csv in get_csv_files(path_test, "full_cont"):
        print(csv)
        basename = os.path.splitext(os.path.basename(csv))[0]
    
        preds_pos = []
        true_pos = []
        
        X_train, y_train = prepare_testing_data_pos_special_delta(csv)   
        for i in range(len(X_train)):
            x_input = X_train[i]

            # Make single prediction and load single groundtruth 
            phat = x_input[-1:]
            ptrue = (0,0,0)

            # Store single prediction and corresponding groundtruth in list
            preds_pos.append(phat)
            true_pos.append(ptrue)
            
        # Create an array 
        preds_pos = array(preds_pos)
        true_pos = array(true_pos)
        true_pos = true_pos.reshape((true_pos.shape[0],1, true_pos.shape[1]))

        # Create dataframe for prediction per timestamp  
        df_preds_pos = pd.DataFrame(np.row_stack(preds_pos), columns= cfg['pos_coords_pred'])
        df_true_pos = pd.DataFrame(np.row_stack(true_pos), columns= cfg['pos_coords_true'])
        df_pos_result = pd.concat([df_preds_pos, df_true_pos], axis=1)

        # Save result to csv
        df_pos_result.to_csv(os.path.join(result_path, basename + '_result_pos.csv'), index=False)
        print("Single Predictions are saved")

In [5]:
########################## 
# Baseline Quaternion 
##########################

def baseline_quat(path_test, result_path, n_features, n_steps, LAT_in_rows):

    for csv in get_csv_files(path_test, "full*contin"):
        print(csv)
        basename = os.path.splitext(os.path.basename(csv))[0]
    
        preds_rot = []
        true_rot = []
        
        X_test, y_test = prepare_testing_data_quat_special(csv)   
        for i in range(len(X_test)):
            x_input = X_test[i]
                
            # Make single prediction and load single groundtruth 
            rhat = x_input[-1]
            rtrue = y_train[i]

            # Store single prediction and corresponding groundtruth in list
            preds_rot.append(yhat)
            true_rot.append(ytrue)
                
        # Create an array 
        preds_rot = array(preds_rot)
        true_rot = array(true_rot)
        true_rot = true_rot.reshape((true_rot.shape[0],1, true_rot.shape[1]))
        
        # Create dataframe for prediction & gt per timestamp and concate results 
        df_preds_rot = pd.DataFrame(np.row_stack(preds_rot), columns= cfg['quat_coords'])
        df_true_rot = pd.DataFrame(np.row_stack(true_rot), columns= cfg['quat_coords'])
        df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)

        # Save result to csv
        df_rot_result.to_csv(os.path.join(result_path, basename + '_result_rot_baseline.csv'), index=False)
        print("Single Predictions are saved")

In [6]:
########################## 
# Baseline Quaternion Delta
##########################

def baseline_quat_delta(path_test, result_path, n_features, n_steps, LAT_in_rows):
    
    
     for csv in get_csv_files(path_test, "full*contin"):
        print(csv)
        basename = os.path.splitext(os.path.basename(csv))[0]
        
        preds_rot = []
        true_rot = []

        X_test, y_test = prepare_testing_data_quat_special_delta(csv)   
        
        for i in range(len(X_test)):
            x_input = X_test[i]

            # Make single prediction and load single groundtruth 
            rhat = x_input[-1:]
            rtrue = (1,0,0,0)

            # Store single prediction and corresponding groundtruth in list
            preds_rot.append(rhat)
            true_rot.append(rtrue)
            
         # Create an array 
        preds_rot = array(preds_rot)
        true_rot = array(true_rot)
        true_rot = true_rot.reshape((true_rot.shape[0],1, true_rot.shape[1]))

        # Create dataframe for prediction per timestamp (preds_rot) and add euler angles 
        df_preds_rot = pd.DataFrame(np.row_stack(preds_rot), columns= cfg['quat_coords_pred'])
        df_true_rot = pd.DataFrame(np.row_stack(true_rot), columns= cfg['quat_coords_true'])
        df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)

        # Save result to csv
        df_rot_result.to_csv(os.path.join(result_path, basename + '_result_delta_rot.csv'), index=False)
        print("Single Predictions are saved")

In [7]:
########################## 
# Neural Net Quaternion 
##########################

def make_prediction_quat(path_test, result_path, n_features, n_steps, LAT_in_rows):

    for csv in get_csv_files(path_test, "full*contin"):
            print(csv)
            basename = os.path.splitext(os.path.basename(csv))[0]

            preds_rot = []
            true_rot = []

            X_test, y_test = prepare_testing_data_quat_special(csv)   
            for i in range(len(X_test)):

                x_input = X_test[i]

                # Reshape x_input to fit tensor
                x_input = x_input.reshape((1, n_steps, n_features))

                # Make single prediction and load single groundtruth 
                rhat = loaded_model.predict(x_input, verbose=0)
                rtrue = y_test[i]

                # Store single prediction and corresponding groundtruth in list
                preds_rot.append(rhat)
                true_rot.append(rtrue)

            # Create an array 
            preds_rot = array(preds_rot)
            true_rot = array(true_rot)
            true_rot = true_rot.reshape((true_rot.shape[0],1, true_rot.shape[1]))

            # Create dataframe for prediction per timestamp (preds_rot) 
            df_preds_rot = pd.DataFrame(np.row_stack(preds_rot), columns= cfg['quat_coords_pred'])
            df_true_rot = pd.DataFrame(np.row_stack(true_rot), columns= cfg['quat_coords_true'])
            df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)

            # Save result to csv
            df_rot_result.to_csv(os.path.join(result_path, basename + '_result_rot.csv'), index=False)
            print("Single Predictions are saved")

In [8]:
########################## 
# Neural Net Quaternion Delta
##########################
      
def make_prediction_delta(path_test, result_path, n_features, n_steps, LAT_in_rows):

    for csv in get_csv_files(path_test, "full*contin"):
            print(csv)
            basename = os.path.splitext(os.path.basename(csv))[0]

            preds_rot = []
            true_rot = []

            X_test, y_test = prepare_testing_data_quat_special_delta(csv)   
            for i in range(len(X_test)):

                x_input = X_test[i]

                # Reshape x_input to fit tensor
                x_input = x_input.reshape((1, n_steps, n_features))

                # Make single prediction and load single groundtruth 
                rhat = loaded_model.predict(x_input, verbose=0)
                rtrue = y_test[i]

                # Store single prediction and corresponding groundtruth in list
                preds_rot.append(rhat)
                true_rot.append(rtrue)

            # Create an array 
            preds_rot = array(preds_rot)
            true_rot = array(true_rot)
            true_rot = true_rot.reshape((true_rot.shape[0],1, true_rot.shape[1]))

            # Create dataframe for prediction per timestamp (preds_rot) and add euler angles 
            df_preds_rot = pd.DataFrame(np.row_stack(preds_rot), columns= cfg['quat_coords_pred'])
            df_true_rot = pd.DataFrame(np.row_stack(true_rot), columns= cfg['quat_coords_true'])
            df_rot_result = pd.concat([df_preds_rot, df_true_rot], axis=1)

            # Save result to csv
            df_rot_result.to_csv(os.path.join(result_path, basename + '_result_rot.csv'), index=False)
            print("Single Predictions are saved")

In [9]:
########################## 
# Neural Net Position  
##########################
        
def make_prediction_pos(path_test, result_path, n_features, n_steps, LAT_in_rows):
     
    for csv in get_csv_files(path_test, "*full*contin*"):
        print(csv)
        basename = os.path.splitext(os.path.basename(csv))[0]
    
        preds_pos = []
        true_pos = []
        
        X_test, y_test = prepare_testing_data_pos_special(csv)   
        for i in range(len(X_test)):
            x_input = X_test[i]

            # Reshape x_input to fit tensor
            x_input = x_input.reshape((1, n_steps, n_features))

            # Make single prediction and load single groundtruth 
            rhat = loaded_model.predict(x_input, verbose=0)
            rtrue = y_test[i]

            # Store single prediction and corresponding groundtruth in list
            preds_pos.append(rhat)
            true_pos.append(rtrue)


        # Create an array 
        preds_pos = array(preds_pos)
        true_pos = array(true_pos)
        true_pos = true_pos.reshape((true_pos.shape[0],1, true_pos.shape[1]))

        # Create dataframe for prediction per timestamp (preds_rot) and add euler angles 
        df_preds_pos = pd.DataFrame(np.row_stack(preds_pos), columns= cfg['pos_coords_pred'])
        df_true_pos = pd.DataFrame(np.row_stack(true_pos), columns= cfg['pos_coords_true'])
        df_pos_result = pd.concat([df_preds_pos, df_true_pos], axis=1)

        # Save result to csv
        df_pos_result.to_csv(os.path.join(result_path, basename + '_result_pos.csv'), index=False)
        print("Single Predictions are saved")

In [10]:
########################## 
# Neural Net Position Delta 
##########################

def make_prediction_delta_pos(path_test, result_path, n_features, n_steps, LAT_in_rows):
     
    for csv in get_csv_files(path_test, "*full*contin"):
        print(csv)
        basename = os.path.splitext(os.path.basename(csv))[0]
    
        preds_pos = []
        true_pos = []
        
        X_test, y_test = prepare_testing_data_pos_special_delta(csv)   
        for i in range(len(X_test)):
            x_input = X_test[i]

            # Reshape x_input to fit tensor
            x_input = x_input.reshape((1, n_steps, n_features))

            # Make single prediction and load single groundtruth 
            rhat = loaded_model.predict(x_input, verbose=0)
            rtrue = y_test[i]

            # Store single prediction and corresponding groundtruth in list
            preds_pos.append(rhat)
            true_pos.append(rtrue)


        # Create an array 
        preds_pos = array(preds_pos)
        true_pos = array(true_pos)
        true_pos = true_pos.reshape((true_pos.shape[0],1, true_pos.shape[1]))

        # Create dataframe for prediction per timestamp (preds_rot) and add euler angles 
        df_preds_pos = pd.DataFrame(np.row_stack(preds_pos), columns= cfg['pos_coords_pred'])
        df_true_pos = pd.DataFrame(np.row_stack(true_pos), columns= cfg['pos_coords_true'])
        df_pos_result = pd.concat([df_preds_pos, df_true_pos], axis=1)

        # Save result to csv
        df_pos_result.to_csv(os.path.join(result_path, basename + '_result_delta_pos.csv'), index=False)
        print("Single Predictions are saved")

# Model Testing

# GRU quat ###############################

In [ ]:
###################################################
# Provide Model ID
model_spec = "Delta_GRU_v01_Neurons_32__DR_0.5__LR_0.01__Batch_128__Epochs_20"
model_ID = cfg['6_1_model_ID']
###################################################

# Result Path 
Path_results_dataset01_GRU_quat_model_id_traces =  Path_results_dataset01_GRU_quat + "/" + model_ID + "/Traces/" 

# Load Model Weights
Path_results_dataset01_GRU_quat_model_id_model_weights = Path_results_dataset01_GRU_quat + model_ID + "/Model/" + model_spec + "_CUDNN_best_model" 

# Load Model Architecture 
Path_results_dataset01_GRU_quat_model_id_model_arch = Path_results_dataset01_GRU_quat + model_ID + "/Model/" + model_spec + "_CUDNN_arch" 


In [ ]:
# load YAML and create model
yaml_file = open(Path_results_dataset01_GRU_quat_model_id_model_arch + '.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)

# load weights into new model
loaded_model.load_weights(Path_results_dataset01_GRU_quat_model_id_model_weights + '.h5')
print("Loaded model from disk")

In [ ]:
###################################################
# Prediction Dataset 01
# Delta
###################################################

make_prediction_delta(Path_dataset01_test, 
                         Path_results_dataset01_GRU_quat_model_id_traces,
                         n_features_quat,
                         n_steps,
                         LAT_in_rows)

In [ ]:
###################################################
# Prediction Dataset 01
# Absolut
###################################################

make_prediction_delta(Path_dataset01_test, 
                         Path_results_dataset01_GRU_quat_model_id_traces,
                         n_features_quat,
                         n_steps,
                         LAT_in_rows)

# GRU pos ###############################

In [ ]:
###################################################
# Provide Model ID
model_spec = "Delta_GRU_v01_Neurons_32__DR_0.1__LR_0.01__Batch_128__Epochs_100"
model_ID = cfg['7_1_model_ID']
###################################################

# Result Path 
Path_results_dataset01_GRU_pos_model_id_traces =  Path_results_dataset01_GRU_pos + "/" + model_ID + "/Traces/" 

# Load Model Weights
Path_results_dataset01_GRU_pos_model_id_model_weights = Path_results_dataset01_GRU_pos + model_ID + "/Model/" + model_spec + "_CUDNN_best_model" 

# Load Model Architecture 
Path_results_dataset01_GRU_pos_model_id_model_arch = Path_results_dataset01_GRU_pos + model_ID + "/Model/" + model_spec + "_CUDNN_arch" 


In [ ]:
# load YAML and create model
yaml_file = open(Path_results_dataset01_GRU_pos_model_id_model_arch + '.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)

# load weights into new model
loaded_model.load_weights(Path_results_dataset01_GRU_pos_model_id_model_weights + '.h5')
print("Loaded model from disk")

In [ ]:
###################################################
# Prediction Dataset 01
# Delta
###################################################
make_prediction_delta_pos(Path_dataset01_test, 
                         Path_results_dataset01_GRU_pos_model_id_traces,
                         n_features_pos,
                         n_steps,
                         LAT_in_rows)

In [ ]:
###################################################
# Prediction Dataset 01
# Absolut
###################################################
make_prediction_pos(Path_dataset01_test, 
                         Path_results_dataset01_GRU_pos_model_id_traces,
                         n_features_pos,
                         n_steps,
                         LAT_in_rows)

# Baseline Quat

In [ ]:
# Load Traces
Path_results_dataset01_Baseline_quat_model_id_traces =  Path_results_dataset01_Baseline_quat + "/Traces/" 

In [ ]:
# Prediction Dataset 01
baseline_quat_delta(Path_dataset01_test,
              Path_results_dataset01_Baseline_quat_model_id_traces,
              n_features_quat,
              n_steps,
              LAT_in_rows)

# Baseline pos

In [11]:
# Load Traces
Path_results_dataset01_Baseline_pos_model_id_traces =  Path_results_dataset01_Baseline_pos + "/Traces/" 

### Delta

In [20]:
# Prediction Dataset 01
baseline_delta_pos(Path_dataset01_test,
               Path_results_dataset01_Baseline_pos_model_id_traces,
               n_features_pos,
               n_steps,
               LAT_in_rows)

C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\1_full_continious.csv
34990
34989
X shape is: (34960, 30, 3)
y shape is: (34960, 3)
------
Single Predictions are saved
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\2_full_continious.csv
14990
14989
X shape is: (14960, 30, 3)
y shape is: (14960, 3)
------
Single Predictions are saved
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\3_full_continious.csv
44990
44989
X shape is: (44960, 30, 3)
y shape is: (44960, 3)
------
Single Predictions are saved
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\4_full_continious.csv
69990
69989
X shape is: (69960, 30, 3)
y shape is: (69960, 3)
------
Single Predictions are saved
C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\5_full_continious.csv
19990
19989
X shape is: (19960, 30, 3)
y shape is: (19960, 3)
------
Single Predictions are saved


### Original

In [17]:
# Prediction Dataset 01
baseline_pos(Path_dataset01_test,
               Path_results_dataset01_Baseline_pos_model_id_traces,
               n_features_pos,
               n_steps,
               LAT_in_rows)

C:/Users/Thomas Weikert/Documents/Thesis/Project_V07_24092021/Data/01_Dataset\1_full_continious.csv
34995
34994
X shape is: (34965, 30, 3)
y shape is: (34965, 3)
------
(34965, 3)


NameError: name 'y_train' is not defined